In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import image_dataset_from_directory, img_to_array
import numpy as np
import matplotlib.pyplot as plt
import os

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Datapath of the training data
TRAIN_DIR = os.path.join(os.getcwd(), 'train')
TEST_DIR = os.path.join(os.getcwd(), 'test')
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128

#policy = tf.keras.mixed_precision.Policy('mixed_float16')
#tf.keras.mixed_precision.set_global_policy(policy)

# Keras has a built in function that can make a tf dataset out of a directory
# The directory should contain subdirectories, each of which contains only one class of images
# The name of these subdirectories will be used as the labels
train_set = image_dataset_from_directory(TRAIN_DIR, labels = 'inferred', color_mode = 'grayscale', batch_size = 512, image_size = (IMAGE_WIDTH, IMAGE_HEIGHT), shuffle = True)

# No labels needed from the test set, so we can simply load up the test directory as an unlabeled dataset
test_set = image_dataset_from_directory(TEST_DIR, labels = 'inferred', color_mode = 'grayscale', batch_size = 512, image_size = (IMAGE_WIDTH, IMAGE_HEIGHT), shuffle = False)

In [ ]:
# Now we need to create a model to train with the training dataset
model = tf.keras.models.Sequential([

    tf.keras.layers.InputLayer(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 1)),
    tf.keras.layers.Rescaling(1.0 / 255),
    tf.keras.layers.Conv2D(32, kernel_size = (3, 3), strides = 2, activation = 'relu'),
    tf.keras.layers.Conv2D(32, kernel_size = (5, 5), strides = 3, activation = 'relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(16, kernel_size = (2, 2), strides = 1, activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2, 2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, kernel_size = (4, 4), strides = 5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(64, activation = 'relu'),
    tf.keras.layers.Dense(2)

])

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = tf.keras.optimizers.Adadelta(learning_rate = 1.0),
    metrics = ['accuracy']
)

model.summary()

In [ ]:
model.fit(
    train_set,
    epochs = 64,
    verbose = 1,
)

In [ ]:
model.evaluate(
    test_set,
    verbose = 1
)